In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import yfinance as yf
import matplotlib.pyplot as plt

In [2]:
class PortfolioOptimization:
    def __init__(self, tickers, start_date, end_date):
        """
        initialize with stock tickers, start date, and end date.
        """
        self.tickers = tickers
        self.start_date = start_date
        self.end_date = end_date
        self.data = None
        self.returns = None
        self.cov_matrix = None
        self.optimized_weights = None

    def fetch_data(self):
        """
        fetch historical stock data from Yahoo Finance.
        """
        self.data = yf.download(self.tickers, start=self.start_date, end=self.end_date)['Adj Close']

    def preprocess_data(self):
        """
        preprocess the data by filling missing values and calculating daily returns.
        """
        self.data.fillna(method='ffill', inplace=True)
        self.data.fillna(method='bfill', inplace=True)
        self.returns = self.data.pct_change().dropna()

    def calculate_metrics(self):
        """
        calculate covariance matrix of returns.
        """
        self.cov_matrix = self.returns.cov()

    def portfolio_return(self, weights):
        """
        calculate portfolio annualized return.
        """
        return np.dot(weights, self.returns.mean()) * 252

    def portfolio_volatility(self, weights):
        """
        calculate portfolio annualized volatility.
        """
        return np.sqrt(np.dot(weights.T, np.dot(self.cov_matrix, weights))) * np.sqrt(252)

    def portfolio_sharpe(self, weights, risk_free_rate=0.02):
        """
        calculate the Sharpe Ratio for the portfolio.
        """
        return (self.portfolio_return(weights) - risk_free_rate) / self.portfolio_volatility(weights)

    def optimize_portfolio(self, risk_free_rate=0.02):
        """
        optimize portfolio weights to maximize Sharpe Ratio.
        """
        num_assets = len(self.tickers)
        initial_weights = np.ones(num_assets) / num_assets
        bounds = [(0, 1) for _ in range(num_assets)]
        constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}

        result = minimize(
            lambda w: -self.portfolio_sharpe(w, risk_free_rate),
            initial_weights,
            method='SLSQP',
            bounds=bounds,
            constraints=constraints
        )

        self.optimized_weights = result.x

    def backtest_portfolio(self):
        """
        backtest the optimized portfolio to calculate cumulative returns.
        """
        portfolio_daily_returns = (self.returns * self.optimized_weights).sum(axis=1)
        cumulative_returns = (1 + portfolio_daily_returns).cumprod()

        #plot cumulative returns
        plt.figure(figsize=(10, 6))
        plt.plot(cumulative_returns, label='Optimized Portfolio')
        plt.title('Cumulative Portfolio Returns')
        plt.xlabel('Date')
        plt.ylabel('Cumulative Returns')
        plt.legend()
        plt.show()

    def visualize_weights(self):
        """
        visualize the portfolio allocation.
        """
        plt.figure(figsize=(10, 6))
        plt.bar(self.tickers, self.optimized_weights, color='skyblue')
        plt.title('Optimized Portfolio Allocation')
        plt.xlabel('Stocks')
        plt.ylabel('Weights')
        plt.show()

    def display_metrics(self):
        """
        portfolio metrics.
        """
        port_return = self.portfolio_return(self.optimized_weights)
        port_volatility = self.portfolio_volatility(self.optimized_weights)
        port_sharpe = self.portfolio_sharpe(self.optimized_weights)

        print(f"Optimized Portfolio Metrics:")
        print(f"Annual Return: {port_return:.2%}")
        print(f"Annual Volatility: {port_volatility:.2%}")
        print(f"Sharpe Ratio: {port_sharpe:.2f}")


In [ ]:
#instance of PortfolioOptimization
tickers = ["UBER", "AMD", "NVDA", "AMZN", "TSLA"]
start_date = "2024-01-01"
end_date = "2025-01-01"

portfolio = PortfolioOptimization(tickers, start_date, end_date)

#fetch and preprocess data
portfolio.fetch_data()
portfolio.preprocess_data()

#calculate metrics and optimize portfolio
portfolio.calculate_metrics()
portfolio.optimize_portfolio()

#results
portfolio.display_metrics()
portfolio.visualize_weights()
portfolio.backtest_portfolio()